In [1]:
import pandas as pd
from neo4j import GraphDatabase

#use your own default url
url = "neo4j://localhost:11007"

#username is neo4j, password is your own password
driver = GraphDatabase.driver(url, auth=("neo4j", "Ima62186Ima@"))

#confirms the connection
def confirmation_query(tx) -> str:
    """
    @param tx is the session that calls the function
    @return result is the string containing the query data
    """
    result = tx.run("CALL db.info()")
    return result.single()

def server_confirmation() -> str:
    """
    @return result is the string containing the info
    """
    with driver.session() as session:
        result = session.write_transaction(confirmation_query)
        return result

server_info = server_confirmation()

print("server info: \n", server_info, '\n', type(server_info))



In [81]:
from neo4j import GraphDatabase
import pandas as pd

class connection:
    def __init__(self) -> None:
        self.driver = GraphDatabase.driver("neo4j://localhost:11007", auth=("neo4j", "Ima62186Ima@"))
        self.df = []
        

    def close(self) -> None:
        self.driver.close()

    #estimation for memory
    @classmethod
    def create_drug_contraints(cls, tx) -> None:
        """
        drug constraint query
        @param cls is the class
        @param tx is the transaction
        @returns none
        """
        query = ("CREATE CONSTRAINT ON (d:Drug) ASSERT d.name IS UNIQUE")
        tx.run(query)


    @classmethod
    def create_category_contraints(cls, tx) -> None:
        """
        category constraint query
        @param cls is the class
        @param tx is the transaction
        @returns none
        """
        query = ("CREATE CONSTRAINT ON (e:Category) ASSERT e.category IS UNIQUE")
        result = tx.run(query)


    @classmethod
    def create_protein_contraints(cls, tx) -> None:
        """
        protein constraint query
        @param cls is the class
        @param tx is the transaction
        @returns none
        """
        query = ("CREATE CONSTRAINT ON (p:Protein) ASSERT p.protein IS UNIQUE")
        result = tx.run(query)


    def run_constraints(self) -> None:
        """
        run all class constraints
        @param self
        @return none
        """
        self.driver.session().write_transaction(self.create_category_contraints)
        self.driver.session().write_transaction(self.create_drug_contraints)
        self.driver.session().write_transaction(self.create_protein_contraints)


    @classmethod
    def create_proteins(cls, tx) -> None:
        """
        creates proteins from caseolap csv
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("LOAD CSV WITH HEADERS FROM 'file:///caseolap.csv' AS row "
                 "CREATE (:Protein {protein: row.protein, CM: row.CM, ARR: row.ARR, CHD: row.CHD, VD: row.VD, IHD: row.IHD, CCS: row.CCS, VOO: row.VOO, OHD: row.OHD, pathways: row.pathways})")
        tx.run(query)


    def run_proteins(self) -> None:
        """
        run the protein creation
        @param self
        @return none
        """
        self.driver.session().write_transaction(self.create_proteins)

    @classmethod
    def create_drug_categories(cls, tx) -> None:
        """
        drug and category node creation
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("LOAD CSV WITH HEADERS FROM 'file:///drug-category-target.csv' AS row "
                 "MERGE (d:Drug {name: row.Name}) "
                 "MERGE (c:Category {category: row.Category})")
        tx.run(query)

    
    def run_drug_categories(self) -> None:
        """
        run drug and category deployment
        @param self
        @return none
        """
        self.driver.session().write_transaction(self.create_drug_categories)

    def get_protein_list(cls, tx) -> any:
        """
        get list of proteins for each drug
        @param self
        @return a data table
        """
        query = ("LOAD CSV WITH HEADERS FROM 'file:///drug-category-target.csv' AS row "
                 "RETURN row.Name as name, row.Category AS category, row.UniprotID AS UniProtID")
        result = tx.run(query)
        return result.data()

    def run_protein_list(self) -> None:
        """
        creates a csv from the data
        @param self
        @return None
        """
        #grab the table from the query
        result = self.driver.session().write_transaction(self.get_protein_list)
        #create dataframe
        result = pd.DataFrame(result)

        #create the string to a list
        for element in range(len(result['UniProtID'])):
            result['UniProtID'][element] = str(result['UniProtID'][element]).split(',')
            for i in range(len(result['UniProtID'][element])):
                result['UniProtID'][element][i] = str(result['UniProtID'][element][i]).strip()
        self.df = result

    @classmethod
    def create_drug_category_relationships(cls, tx, drug, category) -> None:
        """
        query for relationships of the drugs and categories
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("MATCH (d:Drug) WHERE d.name = $drug " 
                 "MATCH (c:Category) WHERE c.category = $category "
                 "MERGE (d)-[r:CATEGORY]->(c)")
        tx.run(query, drug = drug, category = category)

    @classmethod
    def create_category_protein_relationships(cls, tx, category, protein) -> None:
        """
        query for relationships of categories and proteins
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("MATCH (c:Category) WHERE c.category = $category "
                 "MATCH (p:Protein) WHERE p.protein = $protein "
                 "MERGE (p)-[r:CANDIDATE_OF]->(c)")
        tx.run(query, category = category, protein = protein)

    def run_relationships(self) -> None:
        """
        runs all relationships
        @param self is the object of the class
        """
        self.run_protein_list()

        for i in range(len(connect.df['category'])):
            self.driver.session().write_transaction(self.create_drug_category_relationships, self.df['name'][i], self.df['category'][i])
            for j in range(len(connect.df['UniProtID'][i])):
                self.driver.session().write_transaction(self.create_category_protein_relationships, self.df['category'][i], self.df['UniProtID'][i][j])


    

            


In [ ]:
#First Load in the data into Import Folder

In [82]:
connect = connection()
#deploys the constraints
connect.run_constraints()

In [83]:
connect = connection()
#deploys the proteins as nodes
connect.run_proteins()

In [84]:
connect = connection()
#deploys the drugs and categories
connect.run_drug_categories()

In [85]:
connect = connection()
#deploys relationships
result = connect.run_relationships()
